In [68]:
from jupyter_dash import JupyterDash

import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_leaflet as dl
import plotly.express as px
import base64

import numpy as np
import pandas as pd

from pymongo import MongoClient

# import plotly.express as px
# import matplotlib.pyplot as plt

###########################
# Data Manipulation / Model
###########################
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelterRepository import AnimalShelterRepository
# FIX ME update with your username and password and CRUD Python module name
username = 'aacuser'
password = 'aacuser'
repo = AnimalShelterRepository(username, password)

# readAll is the method name from AnimalShelter class and returns a cursor object
# update the next line to match your method name from your CRUD Python module class
query = repo.query({}, 1, 100000)
for record in query:
    del record['_id'] # it doesn't like the ObjectId object.
    del record['outcome_type'] # I don't like to know when the animals are euthanized.
    del record['outcome_subtype'] # Not really relevant.
df = pd.DataFrame.from_records(query)

#########################
# Dashboard Layout / View
#########################

app = JupyterDash('Dash DataTable Only')

image_filename = 'GraziosoSalvareLogo.png' # customer image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard', style={'fontFamily': 'helvetica'}))),
    html.A(
        # customer image location, can you put it in different location and resize?
        html.Img(id='customer-image', src='data:image/png;base64,{}'.format(encoded_image.decode()), alt='customer image', height=80),
        href='https://www.snhu.edu',
        style={
            'position': 'absolute',
            'top': '0',
            'left': '0',
        },
    ),
    html.Div('developed by Mark Holden',
        style={
            'position': 'absolute',
            'top': '55px',
            'right': '5px',
            'fontFamily': 'helvetica',
        }
    ),
    html.Hr(),
    # Button location at top
    html.Div(className='row',
        style={'display' : 'flex'},
        children=[
            html.Button(id='button-water', n_clicks=0, children='Water Rescue'),
            html.Button(id='button-mountain', n_clicks=0, children='Mountain or Wilderness Rescue'),
            html.Button(id='button-tracking', n_clicks=0, children='Disaster Rescue or Individual Tracking'),
            html.Button(id='button-clear', n_clicks=0, children='Clear Filter')
        ]),
    dash_table.DataTable(
        id='data-table-id',
        columns=[
            {'name': i, 'id': i, 'deletable': False, 'selectable': True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action='native',
        sort_action='native',
        sort_mode='multi',
        column_selectable=False,
        row_selectable='multi',
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action='native',
        page_current= 0,
        page_size= 10,
    ),
    html.Div(className='row',
        style={ 'display' : 'flex' },
        children=[
            html.Div([
                    html.H4('Prevalence by Breed'),
                    dcc.Graph(id='pie-id'),
                ],
                style={ 'fontFamily': 'helvetica', 'width': '890px' }
            ),
            html.Div(id='map-id', className='col s12 m6', children=[]),
        ]
    ),
    html.Br(),
    html.Hr(),
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback([Output('data-table-id','filter_query'),
    Output('button-water', 'n_clicks'),
    Output('button-mountain', 'n_clicks'),
    Output('button-tracking', 'n_clicks'),
    Output('button-clear', 'n_clicks')], 
    [Input('button-water', 'n_clicks'),
    Input('button-mountain', 'n_clicks'),
    Input('button-tracking', 'n_clicks'),
    Input('button-clear', 'n_clicks')])
def on_click(waterButton, mountainButton, trackingButton, clearButton):
    if (int(waterButton) > 0):
        return '{animal_type} = Dog\
            && ({breed} = "Labrador Retriever Mix" || {breed} = "Chesapeake Bay Retriever" || {breed} = Newfoundland)\
            && {sex_upon_outcome} = "Intact Female"\
            && {age_upon_outcome_in_weeks} > 26 && {age_upon_outcome_in_weeks} < 156', 0, 0, 0, 0
    if (int(mountainButton) > 0):
        return '{animal_type} = Dog\
            && ({breed} = "German Shepherd" || {breed} = "Alaskan Malamute" || {breed} = "Old English Sheepdog" || {breed} = "Siberian Husky" || {breed} = Rottweiler)\
            && {sex_upon_outcome} = "Intact Male"\
            && {age_upon_outcome_in_weeks} > 26 && {age_upon_outcome_in_weeks} < 156', 0, 0, 0, 0
    if (int(trackingButton) > 0):
        return '{animal_type} = Dog\
            && ({breed} = "Doberman Pinscher" || {breed} = "German Shepherd" || {breed} = "Golden Retriever" || {breed} = Bloodhound || {breed} = Rottweiler)\
            && {sex_upon_outcome} = "Intact Male"\
            && {age_upon_outcome_in_weeks} > 20 && {age_upon_outcome_in_weeks} < 300', 0, 0, 0, 0
    return '', 0, 0, 0, 0

center = [30.75,-97.48] # Austin TX is at [30.75,-97.48]

@app.callback(
    Output('map-id', 'children'),
    [Input('data-table-id', 'derived_viewport_data'),
    Input('data-table-id', 'derived_viewport_selected_rows')])
def update_map(viewData, selected):
    if (selected is None):
        selected = [0]
    dff = pd.DataFrame.from_dict(viewData)

    children = [ dl.TileLayer(id='base-layer-id') ]
    if (dff is not None):
        # Marker with tool tip and popup
        marker = [dl.Marker(position=[dff.iloc[i,11], dff.iloc[i,12]], children=[
                dl.Tooltip(dff.iloc[i,4]),
                dl.Popup([ html.H1(dff.iloc[i,9]) ])
            ]) for i in selected]

        for i in range(len(marker)):
            children.append(marker[i])

    return [ dl.Map(style={'width': '1000px', 'height': '500px'}, center=center, zoom=10, children=children) ]

@app.callback(
    Output('pie-id', 'figure'),
    [Input('data-table-id', 'derived_virtual_data')])
def update_pie(viewData):
    df = pd.DataFrame.from_dict(viewData)
    if (df is not None):
        value_counts = df['breed'].value_counts()
        total_count = sum(value_counts.values)
        idx = np.flatnonzero(value_counts.values > total_count / 100)
        values = np.array(value_counts.values[idx])
        other_value = total_count - sum(values)
        values = np.append(values, other_value)
        names = np.array(value_counts.index[idx])
        names = np.append(names, 'Other')

        fig = px.pie(df, values=values, names=names)
    return fig

app.run_server(mode='external')


Dash app running on http://127.0.0.1:8050/
